In [8]:
import requests
import pandas as pd
import os

In [9]:
download_dir = "./downloads"
os.makedirs(download_dir, exist_ok=True)

In [12]:
base_url = "https://reder-test-o5k8.onrender.com"

In [13]:
# 3. 災害タイプで「Earthquake」のみ抽出
events = requests.get(f"{base_url}/get_events").json()
target_events = [e for e in events if e['disaster_type'].lower() == 'earthquake']
target_events

[{'name': 'Earthquake in Central Tajikistan',
  'description': '2025-04-13: Earthquake in Central Tajikistan on 13 April, 2025',
  'disaster_type': 'Earthquake',
  'country': 'Tajikistan',
  'country_iso3': 'TJK',
  'occurrence_date': '2025-04-13',
  'sa_activation_date': '2025-04-13',
  'requester': 'Central Asian Institute of Applied Geosciences (CAIAG)',
  'escalation_to_charter': 'No',
  'glide_number': 'EQ-2025-000049-TJK',
  'additional_metadata': None,
  'files': [{'name': 'EOS-RS Damage Proxy Map:Tajikistan and Afghanistan,EarthquakesAs Observed by Sentinel-1 image 16 April 2025, v0.7',
    'description': None,
    'url': 'https://sentinel-asia.org/EO/2025/article20250413TJ/EOS-RS_20250416_DPM_S1/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7_MAIN.png',
    'file_type': 'png'},
   {'name': 'EOS-RS Damage Proxy Map:Tajikistan and Afghanistan,EarthquakesAs Observed by Sentinel-1 image 16 April 2025, v0.7',
    'description': None,
    'url': 'https://sentinel-asia

In [14]:
product_list = []
for event in target_events:
    if not event.get('url'):
        continue
    products_url = f"{base_url}/get_products"
    res = requests.get(products_url, params={"url": event['url']})
    if res.status_code != 200:
        continue
    products = res.json()
    kmz_products = [p for p in products if p.get('file_type') == 'kmz']
    for prod in kmz_products:
        product_list.append({
            "event_name": event['name'],
            "product_title": prod.get('title'),
            "download_url": prod.get('download_url')
        })
df_products = pd.DataFrame(product_list)

In [15]:
first_row = df_products.iloc[0]
url = first_row['download_url']
filename = url.split('/')[-1]
save_path = os.path.join(download_dir, filename)

In [16]:
try:
    print(f"Downloading: {url}")
    r = requests.get(url)
    r.raise_for_status()
    with open(save_path, 'wb') as f:
        f.write(r.content)
    print(f"Saved: {save_path}")
except Exception as e:
    print(f"Failed to download {url}: {e}")

Downloading: https://sentinel-asia.org/EO/2025/article20250413TJ/EOS-RS_20250416_DPM_S1/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz
Saved: ./downloads/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz


In [17]:
# ファイルが本当に保存されたか確認
if os.path.exists(save_path):
    print(f"ファイルが正常に保存されました: {save_path}")
else:
    print("ファイルが保存されていません。")

ファイルが正常に保存されました: ./downloads/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz
